In [15]:
import numpy as np
import pandas as pd
import rao_utilities as util
import rao_plotting as plot
import rao_models as model
import rao_stats as stat
import rao_math as m
import warnings
import importlib

from numpy import sqrt
from feets import ExtractorWarning
from pathlib import PurePath

importlib.reload(util)
importlib.reload(plot)
importlib.reload(model)
importlib.reload(m)
importlib.reload(stat)

## CONSTANTS ============================================================================
# FILENAME = 'data/data.csv'
FILENAME = 'data/M3_raw01_Photometry25-47-59.csv' 
# FILENAME = 'data/M3_raw02_Photometry25-47-59.csv'
# FILENAME = 'data/M3_raw03_Photometry28-47-55.csv'
# FILENAME = 'data/M3_raw04_Photometry35-47-55.csv'
# FILENAME = 'data/M3_raw01_Photometry28_47_55_may_29.csv'
# FILENAME = 'data/M3_2nights_rawPhotometry.csv'
# FILENAME = 'data/M3_night_1.xlsx'
# FILENAME = 'data/M3_night_2.xlsx'
EXPOSURE_TIME = 2.5

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) # Need this to prevent it from spamming
warnings.filterwarnings("ignore", category=ExtractorWarning)
## FUNCTIONS ============================================================================

# I moved everything into different python modules since this was getting extremely large

## CALCULATION ============================================================================
file = PurePath(FILENAME)

df = util.extract_data(FILENAME)
bad_rows = df[df['name'] == 'M3-12']
df = df.drop(index=bad_rows.index)

mags = df['mag'].to_numpy(dtype='float64')
error = df['error'].to_numpy(dtype='float64')


if 'jd' in df.columns:
    timeline = df['jd'].unique()
    df['jd'] = pd.to_datetime(df['jd'], origin='julian', unit='D')
else: 
    timeline = np.linspace(0, samples*EXPOSURE_TIME, samples)

average_diff_mags, average_uncertainties = m.calculate_differential_photometry(magnitudes = df['mag'],
                                                                                error = df['error'],
                                                                                num_stars = num_stars,
                                                                                num_samples = num_samples)

df['average_diff_mags'] = average_diff_mags.T.reshape(samples*num_stars, 1)
df['average_uncertainties'] = average_uncertainties.T.reshape(samples*num_stars, 1)

# box_model_results = model.box_least_squares(average_diff_mags, average_uncertainties, timeline=timeline)

# # Feature Extraction information
# indices = m.calculate_all_feets_indices(data = average_diff_mags, 
#                                           timeline = timeline, 
#                                           uncertainties= average_uncertainties)
# feature_df = pd.DataFrame(indices)
# feature_df.to_excel('Features_of_stars.xlsx')

# # Data export for Dr. Langill
# indices_of_stars = [ sub['name'] for sub in box_model_results]
# original_data_filtered = df.iloc[indices_of_stars]
# original_data_filtered.to_excel('Stars_with_varying_light.xlsx')

# # Chi squared test
chi = np.array(stat.reduced_chi_square(mags.T, error.T, timeline))
np.set_printoptions(precision=3, suppress=True, threshold=1000)
print(chi)

# pd.DataFrame({'chisquared':chi}).to_excel('chi_squared_test.xlsx')

# plot.plot_and_save_all(df, file.stem)

[76.742 37.473  6.249 10.788 84.189 52.499  9.716  4.161  4.325  2.289
 19.283 42.949  4.044  6.08   6.282 10.101  3.48   4.614  3.011  3.554
  3.921  4.173  2.721  5.962  4.536  5.074  8.723  5.55   5.312  5.136
  4.519  4.475  3.931  2.627  1.886 10.654  3.416  4.904  4.931  5.441
 14.19   4.432  6.482 16.527  5.307  5.428  5.613  3.737  6.012  3.171
  8.232  3.707  3.329  4.515  4.324  9.311  6.029  4.378  4.849  5.997
  3.058 11.131  3.766  5.651 10.702  4.4    4.774  3.532  5.176 10.934
  2.991  3.232  6.079  3.933  4.582  8.283  3.302]
